In [1]:
import pprint
import time

import zmq
from sympy.physics.units import current

ctx = zmq.Context()
# The REQ talks to Pupil remote and receives the session unique IPC SUB PORT
socket = ctx.socket(zmq.REQ)

ip = 'localhost'
port = 50020

socket.connect(f'tcp://{ip}:{port}')

# Request 'SUB_PORT' for reading data
socket.send_string('SUB_PORT')
sub_port = socket.recv_string()

# Request 'PUB_PORT' for writing data
socket.send_string('PUB_PORT')
pub_port = socket.recv_string()

socket.close()

In [2]:
def create_socket(ctx_c, ip_c, port_sub, topics):
    sub = ctx_c.socket(zmq.SUB)
    sub.connect(f'tcp://{ip_c}:{port_sub}')
    for topic in topics:
        sub.subscribe(topic)
    return sub

def get_object_in_gaze(gaze, r):
    for result in r:
        boxes = result.boxes
        classes = boxes.cls
        for i, box in enumerate(boxes.xyxyn):
            # Check if the gaze point lies within the bounding box of detected objects
            if box[0] <= gaze[0] <= box[2] and box[1] <= (1 - gaze[1]) <= box[3]:
                return result.names[int(classes[i])]
    return None

In [3]:
import requests

def set_lamp_brightness(mode):
    # URL for the Thing Description (base URL provided in the TD itself)
    td_url = "http://10.2.2.33:1880/r402/theglobe"

    try:
        # Send a GET request to retrieve the Thing Description
        response = requests.get(td_url)

        # get the current brightness of the lamp
        curr_brighness = response.json()["brightness"]
        lamp_brighness = curr_brighness

        # if mode is increase, increase the brightness by 10 if it is less than 100
        if mode == "up":
            if curr_brighness <= 80:
                lamp_brighness += 20
            else:
                lamp_brighness = 100
        # if mode is decrease, decrease the brightness by 10 if it is greater than 0
        elif mode == "down":
            if curr_brighness >= 20:
                lamp_brighness -= 20
            else:
                lamp_brighness = 0

        # Check if the request was successful
        if response.status_code == 200:
            # Define the payload to turn the lamp off
            payload = {
                "brightness": lamp_brighness
            }

            # Send the PUT request to change the state of the lamp
            try:
                response = requests.put(td_url, json=payload)
                if response.status_code == 200:
                    print(f"Lamp brightness has been set to {lamp_brighness}.")
                else:
                    print(f"Failed to turn off the lamp. Status code: {response.status_code}")
            except requests.exceptions.RequestException as e:
                print(f"An error occurred: {e}")
        else:
            print(f"Failed to retrieve Thing Description. Status code: {response.status_code}")
    except requests.RequestException as e:
        print(f"An error occurred: {e}")

def set_lamp_color(mode):
    # URL for the Thing Description (base URL provided in the TD itself)
    td_url = "http://10.2.2.33:1880/r402/theglobe"
    colors = ["yellow", "red", "green", "blue", "magenta"]

    try:
        # Send a GET request to retrieve the Thing Description
        response = requests.get(td_url)

        # get the current color of the lamp
        curr_color = response.json()["color"]
        print(curr_color)
        if curr_color in colors:
            curr_index = colors.index(curr_color)
        else:
            curr_index = 0

        # if mode is increase, increase the brightness by 10 if it is less than 100
        if mode == "left":
            if curr_index == 0:
                curr_index = len(colors) - 1
            else:
                curr_index = curr_index - 1
        # if mode is decrease, decrease the brightness by 10 if it is greater than 0
        elif mode == "right":
            if curr_index == len(colors) - 1:
                curr_index = 0
            else:
                curr_index = curr_index + 1

        # Check if the request was successful
        if response.status_code == 200:
            # Define the payload to turn the lamp off
            payload = {
                "color": colors[curr_index]
            }

            # Send the PUT request to change the state of the lamp
            try:
                response = requests.put(td_url, json=payload)
                if response.status_code == 200:
                    print(f"Lamp color has been set to {colors[curr_index]}.")
                else:
                    print(f"Failed to set lamp color. Status code: {response.status_code}")
            except requests.exceptions.RequestException as e:
                print(f"An error occurred: {e}")
        else:
            print(f"Failed to retrieve Thing Description. Status code: {response.status_code}")
    except requests.RequestException as e:
        print(f"An error occurred: {e}")

In [6]:
import numpy as np
import msgpack
from Examples.src.MovementTracker import MovementTracker
import cv2
from ultralytics import YOLO
from PIL import Image

ctx = zmq.Context()
requester = ctx.socket(zmq.REQ)
requester.connect('tcp://localhost:50020') #change ip if using remote machine

# request 'SUB_PORT' for reading data
requester.send_string('SUB_PORT')
ipc_sub_port = requester.recv_string()

socket = create_socket(ctx, 'localhost', sub_port, ['fixation', 'frame.world'])

model = YOLO("../.local/models/object_detection/yolov11n_trained.pt", verbose=False)

last_frame = None

stop_con = False


while not stop_con:
    payload = socket.recv_multipart()
    topic = payload[0].decode("utf-8")

    if topic == 'frame.world':
        last_frame = cv2.imdecode(np.frombuffer(payload[2], dtype=np.uint8), cv2.IMREAD_COLOR)
        last_frame = cv2.cvtColor(last_frame, cv2.COLOR_BGR2RGB)
        last_frame = Image.fromarray(last_frame)

    if topic == 'fixations' and last_frame is not None:
        print("Fixation detected")
        fixation = msgpack.unpackb(payload[1], raw=False)
        fixation = fixation['norm_pos']

        obj_detected = model(last_frame, verbose=False)
        obj = get_object_in_gaze(fixation, obj_detected)
        if obj and "HueLamp" in obj:
            print("HueLamp detected")
            tracker = MovementTracker(fixation, threshold=0.05, min_move_dist=[0.1, 0.15], epsilon=0.1)
            socket.close()
            socket = create_socket(ctx, 'localhost', sub_port, ['gaze.'])
            while True:
                norm_pos = []
                for i in range(5):
                    topic, payload = socket.recv_multipart()
                    message = msgpack.loads(payload)
                    norm_pos.append(message['norm_pos'])

                norm_pos = np.array(norm_pos).mean(axis=0)
                result = tracker.update(norm_pos)

                if result is None:
                    break

                if result and result['returned'] == True:
                    print("Gesture Detected: ", result['direction'])
                    if "up" == result['direction']:
                        set_lamp_brightness(result['direction'])
                    elif "down" == result['direction']:
                        set_lamp_brightness(result['direction'])
                    elif "left" == result['direction']:
                        set_lamp_color(result['direction'])
                    elif "right" == result['direction']:
                        set_lamp_color(result['direction'])
                    break
            socket.close()
            socket = create_socket(ctx, 'localhost', sub_port, ['fixation', 'frame.world'])


Fixation detected
Fixation detected
Fixation detected
Fixation detected
Fixation detected
Fixation detected
Fixation detected
Fixation detected
Fixation detected
HueLamp detected
Gesture Detected:  right
violet
Lamp color has been set to red.
Fixation detected
HueLamp detected
Gesture Detected:  right
red
Lamp color has been set to green.
Fixation detected
HueLamp detected
Gesture Detected:  up
Lamp brightness has been set to 80.
Fixation detected
HueLamp detected
Gesture Detected:  down
Lamp brightness has been set to 60.
Fixation detected
HueLamp detected
Gesture Detected:  left
green
Lamp color has been set to red.
Fixation detected
HueLamp detected
Gesture Detected:  left
red
Lamp color has been set to yellow.
Fixation detected
HueLamp detected
Gesture Detected:  left
gold
Lamp color has been set to magenta.
Fixation detected
HueLamp detected
Gesture Detected:  up
Lamp brightness has been set to 80.
Fixation detected
HueLamp detected
Gesture Detected:  down
Lamp brightness has been


KeyboardInterrupt



In [21]:
import requests

# Define the URL of the lamp
url = "http://10.2.2.33:1880/r402/theglobe"

# Define the payload to turn the lamp off
payload = {
    "on": True,  # Turn the lamp off
    "brightness": 50
}

# Send the PUT request to change the state of the lamp
try:
    response = requests.put(url, json=payload)
    print(response)
    if response.status_code == 200:
        print("The lamp has been turned on successfully.")
    else:
        print(f"Failed to turn off the lamp. Status code: {response.status_code}")
except requests.exceptions.RequestException as e:
    print(f"An error occurred: {e}")

# Sent a GET request to retrieve the Thing Description
response = requests.get(url)
print(response)

<Response [200]>
The lamp has been turned on successfully.
<Response [200]>
